In [22]:
#using DataFrames
using Flux
using Flux.Data: DataLoader
using Flux.Optimise: Optimiser, WeightDecay
using Flux: onehotbatch, onecold, logitcrossentropy

using MLDatasets
using Statistics
using CUDAapi
using Images,ImageView,TestImages,Colors,ImageMagick
using ProgressMeter
using CuArrays

In [23]:
use_cuda = CUDAapi.has_cuda_gpu()
if use_cuda
    device = gpu
    println("Training on GPU")
else
    device = cpu
    println("Training on CPU")
end

Training on GPU


In [102]:
η = 1e-4                  # learning rate
λ = 1e-4                  # L2 regularizer param, implemented as weight decay
epochs = 200              # number of epochs
batchsize = 16            # batch size
input_shape = (224, 224, 3) # input shape for model
aug_multiple = 3          # data augmentation

3

In [103]:
img = load("C:/Users/s7070/monkey/training/training/n0/n0018.jpg")
monk = channelview(img)
monkey = permutedims(monk,(2,3,1))
f_monkey = imresize(monkey,(224,224,3))

224×224×3 Array{N0f8,3} with eltype Normed{UInt8,8}:
[:, :, 1] =
 0.412  0.392  0.427  0.494  0.533  …  0.859  0.714  0.851  0.863  0.541
 0.416  0.392  0.412  0.467  0.51      0.655  0.69   0.89   0.725  0.553
 0.416  0.4    0.396  0.435  0.482     0.412  0.792  0.898  0.78   0.463
 0.412  0.404  0.388  0.42   0.471     0.482  0.694  0.914  0.863  0.498
 0.408  0.408  0.388  0.427  0.471     0.737  0.412  0.906  0.557  0.863
 0.42   0.412  0.412  0.424  0.459  …  0.729  0.494  0.486  0.788  0.918
 0.42   0.427  0.412  0.416  0.427     0.573  0.588  0.365  0.886  0.906
 0.431  0.42   0.416  0.408  0.392     0.463  0.659  0.384  0.812  0.906
 0.439  0.42   0.416  0.4    0.38      0.478  0.725  0.639  0.549  0.937
 0.427  0.443  0.42   0.404  0.38      0.439  0.631  0.816  0.329  0.875
 0.463  0.42   0.42   0.412  0.42   …  0.424  0.612  0.827  0.557  0.663
 0.478  0.404  0.416  0.431  0.451     0.427  0.553  0.8    0.8    0.392
 0.506  0.42   0.404  0.451  0.463     0.443  0.514  0.776 

In [104]:
a = Int64[]
train_dir ="C:/Users/s7070/monkey/training/training"
for i in 1:length(readdir(train_dir,join=true))
    for j in readdir(readdir(train_dir,join=true)[i],join = true)
        img = load(j)
        monk = channelview(img)
        monkey = permutedims(monk,(2,3,1))
        monkey = imresize(monkey,(224,224,3))
        f_monkey = cat(dims=4,f_monkey,monkey)
        append!(a,i)
    end
end 

In [105]:
train_monkey = f_monkey[:,:,:,2:1099];
size(train_monkey)

(224, 224, 3, 1098)

In [106]:
img = load("C:/Users/s7070/monkey/validation/validation/n0/n000.jpg")
t_monk = channelview(img)
t_monkey = permutedims(t_monk,(2,3,1))
tt_monkey = imresize(t_monkey,(224,224,3))

224×224×3 Array{N0f8,3} with eltype Normed{UInt8,8}:
[:, :, 1] =
 0.427  0.435  0.439  0.396  0.38   …  0.259  0.282  0.329  0.392  0.447
 0.447  0.443  0.443  0.404  0.384     0.259  0.298  0.333  0.388  0.447
 0.455  0.447  0.443  0.416  0.4       0.255  0.298  0.345  0.373  0.431
 0.467  0.451  0.447  0.435  0.42      0.251  0.29   0.353  0.388  0.431
 0.478  0.467  0.475  0.475  0.447     0.251  0.286  0.349  0.384  0.443
 0.482  0.482  0.502  0.498  0.478  …  0.251  0.29   0.333  0.38   0.451
 0.502  0.506  0.522  0.522  0.51      0.243  0.286  0.329  0.38   0.451
 0.522  0.533  0.545  0.541  0.529     0.243  0.278  0.329  0.38   0.447
 0.553  0.565  0.565  0.569  0.565     0.251  0.282  0.337  0.38   0.451
 0.565  0.584  0.58   0.573  0.576     0.251  0.286  0.341  0.388  0.455
 0.569  0.58   0.576  0.565  0.588  …  0.255  0.29   0.341  0.38   0.455
 0.565  0.573  0.576  0.588  0.612     0.251  0.282  0.329  0.376  0.451
 0.553  0.561  0.58   0.596  0.624     0.263  0.298  0.341 

In [107]:
b = Int64[]
test_dir ="C:/Users/s7070/monkey/validation/validation"
for i in 1:length(readdir(test_dir,join=true))
    for j in readdir(readdir(test_dir,join=true)[i],join = true)
        img = load(j)
        t_monk = channelview(img)
        t_monkey = permutedims(t_monk,(2,3,1))
        t_monkey = imresize(t_monkey,(224,224,3))
        tt_monkey = cat(dims=4,tt_monkey,t_monkey)
        append!(b,i)
    end
end 

In [108]:

test_monkey = tt_monkey[:,:,:,1:272];
size(test_monkey)

(224, 224, 3, 272)

In [109]:
train_X = convert(Array{Float32,4},train_monkey) 
test_X = convert(Array{Float32,4},test_monkey)
train_y = onehotbatch(a,1:10)
test_y = onehotbatch(b,1:10)

10×272 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

In [110]:
batchsize = 32
train_loader = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)|> device
test_loader = DataLoader(test_X, test_y, batchsize=batchsize)|> device

DataLoader((Float32[0.42745098 0.43529412 … 0.39215687 0.44705883; 0.44705883 0.44313726 … 0.3882353 0.44705883; … ; 0.19607843 0.18431373 … 0.5411765 0.5294118; 0.19215687 0.18431373 … 0.5372549 0.5176471]

Float32[0.6431373 0.627451 … 0.45490196 0.50980395; 0.67058825 0.654902 … 0.4509804 0.5176471; … ; 0.46666667 0.45882353 … 0.7764706 0.7529412; 0.45882353 0.45882353 … 0.77254903 0.7411765]

Float32[0.46666667 0.47058824 … 0.5058824 0.5686275; 0.4627451 0.44705883 … 0.49411765 0.5686275; … ; 0.03529412 0.007843138 … 0.011764706 0.003921569; 0.043137256 0.011764706 … 0.011764706 0.0]

Float32[0.42745098 0.43529412 … 0.39215687 0.44705883; 0.44705883 0.44313726 … 0.3882353 0.44705883; … ; 0.19607843 0.18431373 … 0.5411765 0.5294118; 0.19215687 0.18431373 … 0.5372549 0.5176471]

Float32[0.6431373 0.627451 … 0.45490196 0.50980395; 0.67058825 0.654902 … 0.4509804 0.5176471; … ; 0.46666667 0.45882353 … 0.7764706 0.7529412; 0.45882353 0.45882353 … 0.77254903 0.7411765]

Float32[0.46666667

In [111]:
# println(train_loader,1)

In [112]:
# println(test_loader)

In [125]:
function buildModel(;input_shape)
    return Chain(
        x -> reshape(x, input_shape..., :),
        Conv((5, 5), 3=>24, stride=2),
        BatchNorm(24, relu),
        Conv((3, 3), 24=>48, stride=2),
        BatchNorm(48, relu),
        Conv((3, 3), 48=>64, stride=2),
        BatchNorm(64, relu),
        Conv((1, 1), 64=>64, stride=1),
        BatchNorm(64, relu),
        Conv((3, 3), 64=>128, stride=2),
        BatchNorm(128, relu),
        Conv((1, 1), 128=>128, stride=1),
        BatchNorm(128, relu),
        GlobalMeanPool(),
        flatten,
        Dropout(0.1),
        Dense(128, 64),
        Dropout(0.1),
        Dense(64, 10),
        softmax
    )
end

model = buildModel(input_shape=input_shape) |> device;
# println("CNNs model: $(num_params(model)) trainable params");

In [126]:
fake_input = randn(input_shape) |> device
fake_output = model(fake_input)
size(fake_output)

(10, 1)

In [127]:
loss(ŷ, y) = logitcrossentropy(ŷ, y)

round4(x) = round(x, digits=4)

function calc_loss_accuracy(loader, model, device)
    l = 0f0
    acc = 0
    ntot = 0
    for (x, y) in loader
        x, y = x |> device, y |> device
        ŷ = model(x)
        l += loss(ŷ, y) * size(x)[end]        
        acc += sum(onecold(ŷ |> cpu) .== onecold(y |> cpu))
        ntot += size(x)[end]
    end
    return (loss = l/ntot |> round4, acc = acc/ntot*100 |> round4)
end

calc_loss_accuracy (generic function with 1 method)

In [128]:
function callback(epoch)
    train = calc_loss_accuracy(train_loader, model, device)
    
    test = calc_loss_accuracy(test_loader, model, device)        
    println("Epoch: $epoch   Train: $(train)   Test: $(test)")
end

callback (generic function with 1 method)

In [129]:
opt = ADAM(η) 
if λ > 0 
    opt = Optimiser(opt, WeightDecay(λ))
end

Optimiser(Any[ADAM(0.0001, (0.9, 0.999), IdDict{Any,Any}()), WeightDecay(0.0001)])

In [130]:

params_model = Flux.params(model)

println("Starting Training")
callback(0)


Starting Training
Epoch: 0   Train: (loss = 2.3025f0, acc = 9.6539)   Test: (loss = 2.3025f0, acc = 9.5588)


In [131]:
for epoch in 1:epochs
    progress = ProgressMeter.Progress(length(train_loader))
    
    # update parameters
    for (x, y) in train_loader
        x = x |> device
        y = y |> device
        grads_model = Flux.gradient(params_model) do
            ŷ = model(x)
            loss(ŷ, y)
        end
        Flux.Optimise.update!(opt, params_model, grads_model)
        ProgressMeter.next!(progress)   # comment out for no progress bar
    end
    
    #  logging
    if epoch % 5 == 0
        callback(epoch)
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


Epoch: 5   Train: (loss = 2.1172f0, acc = 38.3424)   Test: (loss = 2.1352f0, acc = 34.9265)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


Epoch: 10   Train: (loss = 2.0534f0, acc = 45.9016)   Test: (loss = 2.0862f0, acc = 41.5441)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


...

Epoch: 190   Train: (loss = 1.5214f0, acc = 97.2678)   Test: (loss = 1.872f0, acc = 63.6029)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


Epoch: 195   Train: (loss = 1.5148f0, acc = 97.6321)   Test: (loss = 1.881f0, acc = 63.2353)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


Epoch: 200   Train: (loss = 1.5039f0, acc = 98.3607)   Test: (loss = 1.883f0, acc = 59.5588)


In [132]:
test = calc_loss_accuracy(test_loader, model, device)        
println("Test: $(test)")

Test: (loss = 1.883f0, acc = 59.5588)
